<a href="https://colab.research.google.com/github/Realmbird/Recreation-Consitutional-AI/blob/main/Reward_model_helpful_Qwen_8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install -U datasets

In [1]:
from datasets import load_dataset

In [4]:
dataset = load_dataset("Anthropic/hh-rlhf", data_dir="helpful-base")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [7]:
dataset["train"][0]

{'chosen': '\n\nHuman: Hi, I want to learn to play horseshoes. Can you teach me?\n\nAssistant: I can, but maybe I should begin by telling you that a typical game consists of 2 players and 6 or 8 horseshoes.\n\nHuman: Okay. What else is needed to play, and what are the rules?\n\nAssistant: A horseshoe is usually made out of metal and is about 3 to 3.5 inches long and around 1 inch thick. The horseshoe should also have a 2 inch by 3 inch flat at the bottom where the rubber meets the metal. We also need two stakes and six horseshoes.',
 'rejected': '\n\nHuman: Hi, I want to learn to play horseshoes. Can you teach me?\n\nAssistant: I can, but maybe I should begin by telling you that a typical game consists of 2 players and 6 or 8 horseshoes.\n\nHuman: Okay. What else is needed to play, and what are the rules?\n\nAssistant: Horseshoes are either metal or plastic discs. The horseshoes come in different weights, and the lighter ones are easier to throw, so they are often the standard for begi

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
model_name = "Qwen/Qwen3-8B" # Or any other suitable model
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Important: Add a pad token if the tokenizer doesn't have one, especially for decoder models.
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2) # accepted and rejceted label
# Resize token embeddings if you added a new pad token
model.resize_token_embeddings(len(tokenizer))

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

...

model-00001-of-00005.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/3.19G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
from trl import RewardTrainer, RewardConfig
from transformers import TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./reward_model_training", # Directory to save checkpoints
    per_device_train_batch_size=4,
    num_train_epochs=1,
    logging_steps=100,
    save_steps=500,
    evaluation_strategy="steps",
    eval_steps=500,
    gradient_accumulation_steps=1,
    learning_rate=2e-5,
    report_to="none"
)

# Configure RewardTrainer
# The RewardTrainer automatically handles the pairwise comparison loss
trainer = RewardTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=dataset["train"], # Use the training split
    eval_dataset=dataset["test"], # Use the test split for evaluation
)

In [ ]:
trainer.train()